In [2]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd

from dm import Country, DemographicModeling
from dm.utils import get_countries, add_enrich_aliases

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy

In [3]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent

dir_data = dir_prj/'data'
dir_raw = dir_data/'raw'
dir_int = dir_data/'interim'

gdb_raw = dir_raw/'raw.gdb'
gdb_int = dir_int/'interim.gdb'

# import the project package from the project package path - only necessary if you are not using a unique environemnt for this project
sys.path.append(str(dir_prj/'src'))
import canada_modeling

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

True

In [9]:
lvl0_fc_pth = gdb_int/'van_lvl0_keyvars'
rona_fc_pth = gdb_int/'biz_rona'
comp_fc_pth =gdb_int/'biz_comp'

for pth in [lvl0_fc_pth, rona_fc_pth, comp_fc_pth]:
    assert arcpy.Exists(str(pth))
    print(pth)

D:\projects\canada-modeling\data\interim\interim.gdb\van_lvl0_keyvars
D:\projects\canada-modeling\data\interim\interim.gdb\biz_rona
D:\projects\canada-modeling\data\interim\interim.gdb\biz_comp


In [5]:
can = Country('CAN')

can

<dm.Country - CAN (local)>

In [6]:
van_df = can.cmacas.get('vancouver')

van_df

,ID,NAME,SHAPE
0,933,Vancouver,"{""rings"": [[[-122.89240540549281, 49.087271790..."


In [35]:
lvl0_df = pd.DataFrame.spatial.from_featureclass(lvl0_fc_pth)
rona_df = pd.DataFrame.spatial.from_featureclass(rona_fc_pth)
comp_df = pd.DataFrame.spatial.from_featureclass(comp_fc_pth)

In [36]:
lvl0_rona_df = lvl0_df.dm.get_nearest(rona_df, source=can, origin_id_column='id', destination_id_column='locnum', destination_count=3, near_prefix='brand')

lvl0_rona_df.head()

,OBJECTID,id,name,key_can_facts_p3_yptapop,key_can_facts_p3_yhfscf,key_can_facts_p3_yhszavg,key_can_facts_p3_yhniavg,key_can_facts_p3_yhnimed,key_can_facts_p3_ytenhhd,key_can_facts_p3_ytenown,...,brand_destination_id_02,brand_proximity_kilometers_02,brand_proximity_minutes_02,brand_proximity_side_street_left_02,brand_proximity_side_street_right_02,brand_destination_id_03,brand_proximity_kilometers_03,brand_proximity_minutes_03,brand_proximity_side_street_left_03,brand_proximity_side_street_right_03
0,1,59150369,59150369,596.0,148.0,2.6,90408.28,71666.67,225.0,116.0,...,47496609,13.898064,20.873741,1.0,0.0,20690194,20.355922,27.358089,1.0,0.0
1,2,59150370,59150370,1498.0,393.0,2.1,93374.01,82450.98,705.0,357.0,...,47496609,13.905167,20.416101,1.0,0.0,41514332,17.956814,27.467442,1.0,0.0
2,3,59150373,59150373,767.0,201.0,3.0,148982.53,117672.41,253.0,199.0,...,20690301,10.584132,21.137046,0.0,1.0,20690202,13.590163,27.913134,1.0,0.0
3,4,59150375,59150375,507.0,130.0,3.0,141579.89,102343.75,167.0,115.0,...,20690301,10.810926,21.809486,0.0,1.0,20690202,14.203826,29.063980,1.0,0.0
4,5,59150376,59150376,497.0,128.0,2.9,130564.61,101973.68,173.0,100.0,...,20690301,11.016347,22.004528,0.0,1.0,20690202,13.922363,28.585160,1.0,0.0


In [37]:
lvl0_prx_df = lvl0_rona_df.dm.get_nearest(comp_df, source=can, origin_id_column='id', destination_id_column='locnum', destination_count=3, near_prefix='comp')

lvl0_prx_df.head()

,OBJECTID,id,name,key_can_facts_p3_yptapop,key_can_facts_p3_yhfscf,key_can_facts_p3_yhszavg,key_can_facts_p3_yhniavg,key_can_facts_p3_yhnimed,key_can_facts_p3_ytenhhd,key_can_facts_p3_ytenown,...,comp_destination_id_02,comp_proximity_kilometers_02,comp_proximity_minutes_02,comp_proximity_side_street_left_02,comp_proximity_side_street_right_02,comp_destination_id_03,comp_proximity_kilometers_03,comp_proximity_minutes_03,comp_proximity_side_street_left_03,comp_proximity_side_street_right_03
0,1,59150369,59150369,596.0,148.0,2.6,90408.28,71666.67,225.0,116.0,...,47496609,13.898064,20.873741,1.0,0.0,20690194,20.355922,27.358089,1.0,0.0
1,2,59150370,59150370,1498.0,393.0,2.1,93374.01,82450.98,705.0,357.0,...,47496609,13.905167,20.416101,1.0,0.0,41514332,17.956814,27.467442,1.0,0.0
2,3,59150373,59150373,767.0,201.0,3.0,148982.53,117672.41,253.0,199.0,...,20690301,10.584132,21.137046,0.0,1.0,20690202,13.590163,27.913134,1.0,0.0
3,4,59150375,59150375,507.0,130.0,3.0,141579.89,102343.75,167.0,115.0,...,20690301,10.810926,21.809486,0.0,1.0,20690202,14.203826,29.063980,1.0,0.0
4,5,59150376,59150376,497.0,128.0,2.9,130564.61,101973.68,173.0,100.0,...,20690301,11.016347,22.004528,0.0,1.0,20690202,13.922363,28.585160,1.0,0.0


In [38]:
lvl0_trn_fc = lvl0_prx_df.spatial.to_featureclass(gdb_int/'van_lvl0_trn')

In [39]:
add_enrich_aliases(lvl0_trn_fc, can)

'D:\\projects\\canada-modeling\\data\\interim\\interim.gdb\\van_lvl0_trn'